In [ ]:
def check_for_proximal_windfarm(idx_2d, distance_to_windfarm = 3000):
    # Assuming your iris cube is named 'monthly_cube'
    # Get the coordinates of the center of the grid cell using the indices provided
    projection_y_coord = one_ts.coord('projection_y_coordinate').points[idx_2d[0]]
    projection_x_coord = one_ts.coord('projection_x_coordinate').points[idx_2d[1]]

    # Convert BNG coordinates of the grid cell to latitude and longitude
    lon_center, lat_center = bng_to_latlon(projection_x_coord, projection_y_coord)

    # Load your wind farms dataset containing 'X-coordinate' and 'Y-coordinate' columns in BNG
    # Assuming your wind farms dataset is named 'wind_farms_df'
    # Iterate over each wind farm location
    for index, row in windfarms.iterrows():
        # Convert BNG coordinates of wind farm location to latitude and longitude
        #lon_farm, lat_farm = bng_to_latlon(row['X-coordinate'], row['Y-coordinate'])
        # Calculate distance between wind farm location and the center of the grid cell
        distance = calculate_distance(lon_center, lat_center, row['lon'], row['lat'])

        # Check if the distance is within 3km
        if distance <= distance_to_windfarm:  # 3km in meters
            print(f"Wind farm at index {index} is within 3km of the grid cell.")
    print("Finished checking for close windfarms")


In [131]:
# windfarm within 5km
# [11,27,35,36,57,61,75,95,97,101,106,112,117,120,132,146,180,186,190,194,204,205,212,226,227,229,232,239,279,285,348,376,393]
# windfarm within 3km
len([27,36,57,61,75,97,101,106,120,132,190,204, 239, 285,348,376])

16

In [35]:
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point
from pyproj import Proj, transform, Geod
import pandas as pd

windfarms = pd.read_csv('/nfs/a319/gy17m2a/PhD/datadir/NIMROD/windfarms.csv', encoding='latin1')
windfarms =windfarms[windfarms['Technology Type']=='Wind Onshore']
windfarms = windfarms[(windfarms['Development Status'] == 'Operational') | (windfarms['Development Status'] == 'Decommissioned')]

# Read shapefile of UK outline
uk_outline = gpd.read_file('/nfs/a319/gy17m2a/PhD/datadir/SpatialData/UK_shp/GBR_adm0.shp')

# Define a function to convert BNG coordinates to latitude and longitude
def bng_to_latlon(x, y):
    in_proj = Proj(init='epsg:27700')  # British National Grid projection
    out_proj = Proj(init='epsg:4326')  # WGS84 (latitude and longitude) projection
    lon, lat = transform(in_proj, out_proj, x, y)
    return lon, lat

# # Read DataFrame with BNG coordinates
df = windfarms

# # Convert BNG coordinates to latitude and longitude
df['lon'], df['lat'] = zip(*df.apply(lambda row: bng_to_latlon(row['X-coordinate'], row['Y-coordinate']), axis=1))

# # Convert DataFrame to GeoDataFrame
# geometry = [Point(lon, lat) for lon, lat in zip(df['lon'], df['lat'])]
# gdf = gpd.GeoDataFrame(df, geometry=geometry, crs='EPSG:4326')

# # Plot the UK outline and points
# fig, ax = plt.subplots(figsize=(10, 10))
# uk_outline.plot(ax=ax, color='lightgrey', edgecolor='black')
# gdf.plot(ax=ax, color='red', markersize=5)
# plt.xlabel('Longitude')
# plt.show()

In [30]:
import iris
import glob
import datetime as datetime
import sys
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from pyproj import Transformer
import itertools
from scipy import spatial
import numpy.ma as ma
import time
import math
import tilemapbase

In [2]:
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)
simplefilter(action='ignore', category=UserWarning)
simplefilter(action='ignore', category=DeprecationWarning)

# Create path to files containing functions
root_fp = "/nfs/a319/gy17m2a/PhD/"
sys.path.insert(0, root_fp + 'Scripts/GlobalFunctions')
from Spatial_plotting_functions import *
from Spatial_geometry_functions import *
gb_gdf = create_gb_outline({'init' :'epsg:3857'})

In [117]:
# Function to calculate distance between two points
def calculate_distance(lon1, lat1, lon2, lat2):
    geod = Geod(ellps='WGS84')
    _, _, distance = geod.inv(lon1, lat1, lon2, lat2)
    return distance

In [10]:
# Define the directory and create a sorted list of file paths
year = 2012
radardir = f'/nfs/a161/gy17m2a/PhD/datadir/NIMROD/5mins/OriginalFormat_1km/{year}/'
file_list = glob.glob(radardir + "*.nc")
sorted_list = sorted(file_list)

# Load the cubes
cubes = iris.load(sorted_list[0])

# Concatenate into a single cube
one_day = cubes.concatenate_cube()

In [99]:
one_ts = one_day[1,:,:]

In [130]:
for gauge_num in [11,27,35,36,57,61,75,95,97,101,106,112,117,120,132,146,180,186,190,194,204,205,212,226,227,229,232,239,279,285,348,376,393]:
    print(gauge_num)
    gauge = tbo_vals.iloc[gauge_num]

    # closest_point, idx_2d = find_position_obs(one_ts,gauge1['Lat'], gauge1['Lon'], plot=True)
    closest_point, idx_2d = find_position_obs_nomasking(one_ts,gauge['Lat'], gauge['Lon'], plot=False)
    check_for_proximal_windfarm(idx_2d, 3000)
    end=time.time()

11
Finished checking for close windfarms
27
Wind farm at index 3074 is within 3km of the grid cell.
Finished checking for close windfarms
35
Finished checking for close windfarms
36
Wind farm at index 3191 is within 3km of the grid cell.
Finished checking for close windfarms
57
Wind farm at index 3155 is within 3km of the grid cell.
Finished checking for close windfarms
61
Wind farm at index 2961 is within 3km of the grid cell.
Wind farm at index 3519 is within 3km of the grid cell.
Finished checking for close windfarms
75
Wind farm at index 2472 is within 3km of the grid cell.
Finished checking for close windfarms
95
Finished checking for close windfarms
97
Wind farm at index 2287 is within 3km of the grid cell.
Finished checking for close windfarms
101
Wind farm at index 3604 is within 3km of the grid cell.
Wind farm at index 3821 is within 3km of the grid cell.
Finished checking for close windfarms
106
Wind farm at index 2287 is within 3km of the grid cell.
Wind farm at index 3594 i

### At each grid cell with a gauge in it, check the NIMROD time series

In [127]:
for year in range(20)
radardir = f'/nfs/a161/gy17m2a/PhD/datadir/NIMROD/5mins/OriginalFormat_1km/{year}/'
file_list = glob.glob(radardir + "*.nc")
sorted_list = sorted(file_list)

# Load the cubes
cubes = iris.load(sorted_list[:30])

# Concatenate into a single cube
monthly_cube = cubes.concatenate_cube()

(590500.0, 397500.0)

In [98]:
# # Get tb0 values at each gauge
# tbo_vals = pd.read_csv('/nfs/a319/gy17m2a/PhD/datadir/RainGauge/interarrival_thresholds_CDD_noMissing.txt')
# gauge1 = tbo_vals.iloc[1]

# # closest_point, idx_2d = find_position_obs(one_ts,gauge1['Lat'], gauge1['Lon'], plot=True)
# closest_point, idx_2d = find_position_obs_nomasking(one_ts,53.399100, -4.387940, plot=True)

In [139]:
# import iris
# import glob
# import iris.plot as iplt
# import iris.quickplot as qplt
# import datetime as datetime
# import iris.coord_categorisation as cat
# import sys
# import numpy as np

# # Get year from input of running the script 
# # year = sys.argv[1]
# # print (year)
# years = [2016]

# for year in years:
#     print(year)
#     ### Get list of files to convert
#     radardir = f'/nfs/a161/gy17m2a/PhD/datadir/NIMROD/5mins/OriginalFormat_1km/{year}/'
#     file_list=glob.glob(radardir +"*.nc")
#     sorted_list = sorted(file_list)
    
#     for i in range(0,len(sorted_list)):
#         print(i)
    

In [160]:
# for year in [2012,2013]:
#     print(year)
#     ### Get list of files to convert
#     radardir = f'/nfs/a161/gy17m2a/PhD/datadir/NIMROD/5mins/OriginalFormat_1km/{year}/'
#     file_list=glob.glob(radardir +"*.nc")
#     sorted_list = sorted(file_list)
    
#     for i in range(0,len(sorted_list)):
#         ### Load radar data for one day (using IRIS)
#         day_cube = iris.load_cube(sorted_list[i])
#         day_cube = day_cube[:,825,891]
#         print(i, ':', np.nanmax(day_cube.data))

cubes = iris.load(sorted_list)
monthly_cube = cubes.concatenate_cube()

In [163]:
my_location = monthly_cube[:,825,891]
test=pd.DataFrame({'data':my_location.data})

In [148]:
### Load radar data for one day (using IRIS)
day_cube = iris.load_cube(sorted_list[i])
day_cube = day_cube[:,825,891]

### Add additional time based variables
# cat.add_year(day_cube, 'time', name='year')
# cat.add_month(day_cube, 'time', name='month')
# cat.add_day_of_month(day_cube, 'time', name='day_of_month')
cat.add_hour(day_cube, 'time', name='hour')
# cat.add_day(day_cube, 'time', name='day')

### Aggregate to half hourly values (means)
firsthalfof_hour_constraint = iris.Constraint(time=lambda cell: cell.point.minute <30)
secondhalfof_hour_constraint = iris.Constraint(time=lambda cell: cell.point.minute >=30)

# Create empty cube list to populate
my_cube_list = iris.cube.CubeList()

# Get list of the hours
hours = set(day_cube.coord('hour').points)

max_vals = []

# Loop through the hours
for hour in hours:
    print(hour)

    # Establish constraint to select only this hour
    hour_constraint = iris.Constraint(time=lambda cell: cell.point.hour == hour)
    # Use constraint to select only this hour
    hour_cube = day_cube.extract(hour_constraint)
    # Check the times
    # times = hour_cube.coord('time').points
    # times = [datetime.datetime.fromtimestamp(x ) for x in times]

    # Get only cubes which fall within the first half of the hour and then the second half of the hour
    first_half_of_hour = hour_cube.extract(firsthalfof_hour_constraint)
    second_half_of_hour = hour_cube.extract(secondhalfof_hour_constraint)

    # If there are at least 4 values
    # Find the mean across first/second halves of hour
    # Add to cube list
    if first_half_of_hour == None:
        print("no values in 1st half hour")
    elif len(first_half_of_hour.shape) ==2:
        print("only 1 value in 1st half hour")        
    else:

        if first_half_of_hour.shape[0] >=4:
            print(f"iter {i}, hour {hour}, first half hour, min value is: {np.nanmin(first_half_of_hour.data)}")
            print(f"iter {i}, hour {hour}, first half hour, max value is: {np.nanmax(first_half_of_hour.data)}")
            ## Correct negative 1064 values to np.nan
            if np.nanmin(first_half_of_hour.data)<0:
                first_half_of_hour.data = np.where(first_half_of_hour.data <0, np.nan, first_half_of_hour.data)
                # If somehow still negatives
                if np.nanmin(first_half_of_hour.data <0):
                    print(first_half_of_hour.data[first_half_of_hour.data<0])
            # Get rid of this val
            first_half_of_hour.data = np.where(first_half_of_hour.data ==1023.96875, np.nan, first_half_of_hour.data)  
            first_half_of_hour.data = np.where(first_half_of_hour.data ==9.969209968386869e+36, np.nan, first_half_of_hour.data)
            print(f"iter {i}, hour {hour}, first half hour, min value is now: {np.nanmin(first_half_of_hour.data)}") 
            print(f"iter {i}, hour {hour}, first half hour, max value is now: {np.nanmax(first_half_of_hour.data)}")  

            # FIND MEAN ACROSS WHOLE FIRST HALF HOUR
            first_half_hourly_mean = first_half_of_hour.aggregated_by(['hour'],iris.analysis.MEAN)
            print(np.nanmax(first_half_hourly_mean.data))
            # first_half_hourly_mean.data.astype('float64')
            my_cube_list.append(first_half_hourly_mean)
        else:
            print(f"only {first_half_of_hour.shape[0]} vals in 1st half hour")

    ### SECOND HALF HOUR    
    if second_half_of_hour == None:
        print("no values in 2nd half hour")
    elif len(second_half_of_hour.shape) ==2:
        print("only 1 value in 2nd half hour")
    else:
        if second_half_of_hour.shape[0] >=4:
            print(f"iter {i}, hour {hour}, second half hour, min value is: {np.nanmin(second_half_of_hour.data)}")
            print(f"iter {i}, hour {hour}, second half hour, max value is: {np.nanmax(second_half_of_hour.data)}")
            ## Correct negative 1064 values to np.nan
            if np.nanmin(second_half_of_hour.data)<0:
                second_half_of_hour.data = np.where(second_half_of_hour.data <0, np.nan, second_half_of_hour.data)
                # If somehow still negatives
                if np.nanmin(second_half_of_hour.data <0):
                    print(second_half_of_hour.data[second_half_of_hour.data<0])
            # Get rid of this val
            second_half_of_hour.data = np.where(second_half_of_hour.data ==1023.96875, np.nan, second_half_of_hour.data)        
            second_half_of_hour.data = np.where(second_half_of_hour.data ==9.969209968386869e+36, np.nan, second_half_of_hour.data)

            print(f"iter {i}, hour {hour}, second half hour, min value is now: {np.nanmin(second_half_of_hour.data)}") 
            print(f"iter {i}, hour {hour}, second half hour, max value is now: {np.nanmax(second_half_of_hour.data)}") 

            max_vals.append(np.nanmax(second_half_of_hour.data))
            max_vals.append(np.nanmax(first_half_of_hour.data))

            # FIND MEAN ACROSS WHOLE FIRST HALF HOUR
            second_half_hourly_mean = second_half_of_hour.aggregated_by(['hour'],iris.analysis.MEAN)
            print(np.nanmax(second_half_hourly_mean.data))
            # second_half_hourly_mean.data.astype('float64')
            my_cube_list.append(second_half_hourly_mean)
        else:
            print(f"only {second_half_of_hour.shape[0]} vals in 2nd half hour")


### Join back into one cube covering the whole day
try:
    for halfhour_i in range(0,len(my_cube_list)):
        my_cube_list[halfhour_i].data = my_cube_list[halfhour_i].data.astype('float64')

    thirty_mins_means = my_cube_list.concatenate_cube()

    # Get rid of high values which are fill values
    thirty_mins_means.data = np.where(thirty_mins_means.data >1e+36, np.nan, thirty_mins_means.data)

    # save 
    new_fp = sorted_list[i][:-3]+ '_30mins.nc'
    new_fp = new_fp.replace('5mins', '30mins')
    iris.save(thirty_mins_means, new_fp)
    print(f'Saved cube {year} {i}')
    print(np.nanmin(thirty_mins_means.data))
    print(np.nanmax(thirty_mins_means.data))
    print(np.nanmean(thirty_mins_means.data))

except:
    pass

0
iter 91, hour 0, first half hour, min value is: 0.0
iter 91, hour 0, first half hour, max value is: 0.0
iter 91, hour 0, first half hour, min value is now: 0.0
iter 91, hour 0, first half hour, max value is now: 0.0
0.0
iter 91, hour 0, second half hour, min value is: 0.0
iter 91, hour 0, second half hour, max value is: 0.0
iter 91, hour 0, second half hour, min value is now: 0.0
iter 91, hour 0, second half hour, max value is now: 0.0
0.0
1
iter 91, hour 1, first half hour, min value is: 0.0
iter 91, hour 1, first half hour, max value is: 0.0
iter 91, hour 1, first half hour, min value is now: 0.0
iter 91, hour 1, first half hour, max value is now: 0.0
0.0
iter 91, hour 1, second half hour, min value is: 0.0
iter 91, hour 1, second half hour, max value is: 0.0
iter 91, hour 1, second half hour, min value is now: 0.0
iter 91, hour 1, second half hour, max value is now: 0.0
0.0
2
iter 91, hour 2, first half hour, min value is: 0.0
iter 91, hour 2, first half hour, max value is: 0.0
it

iter 91, hour 18, second half hour, min value is: 0.0
iter 91, hour 18, second half hour, max value is: 0.0
iter 91, hour 18, second half hour, min value is now: 0.0
iter 91, hour 18, second half hour, max value is now: 0.0
0.0
19
iter 91, hour 19, first half hour, min value is: 0.0
iter 91, hour 19, first half hour, max value is: 0.0
iter 91, hour 19, first half hour, min value is now: 0.0
iter 91, hour 19, first half hour, max value is now: 0.0
0.0
iter 91, hour 19, second half hour, min value is: 0.0
iter 91, hour 19, second half hour, max value is: 0.0
iter 91, hour 19, second half hour, min value is now: 0.0
iter 91, hour 19, second half hour, max value is now: 0.0
0.0
20
iter 91, hour 20, first half hour, min value is: 0.0
iter 91, hour 20, first half hour, max value is: 0.0
iter 91, hour 20, first half hour, min value is now: 0.0
iter 91, hour 20, first half hour, max value is now: 0.0
0.0
iter 91, hour 20, second half hour, min value is: 0.0
iter 91, hour 20, second half hour, 